In [1]:
import os


In [3]:
%pwd

'd:\\end to end 2\\Chicken-Disease-end-to-end-\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\end to end 2\\Chicken-Disease-end-to-end-'

In [52]:
#first prepapring confgi yaml
#Entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_dir: Path
    checkpoint_model_path: Path

    



In [53]:
from cnnClassfier.constants import *
from cnnClassfier.utils.common import read_yaml, create_directories


In [54]:
class ConfigurtaionManager:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_prepare_callbacks_config(self) ->PrepareCallbacksConfig:
        config= self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.check_point_model_path)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_dir)

        ])
        prepare_callback_config= PrepareCallbacksConfig(
         root_dir= Path(config.root_dir),
         tensorboard_root_dir= Path(config.tensorboard_root_dir),
         checkpoint_model_path=config.check_point_model_path
            )
        return prepare_callback_config


In [55]:
config=ConfigurtaionManager
print(config.get_prepare_callbacks_config)


<function ConfigurtaionManager.get_prepare_callbacks_config at 0x00000211CF281E10>


In [49]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf 
import time

In [58]:
class PrepareCallback:
    def __init__(self, config : PrepareCallbacksConfig):
        self.config= config
        

    @property
    def _create_tb_callbacks(self):
            timestamp= time.strftime("%Y-%m-d-%H-%M-%S")
            tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_dir,
            f"tb_logs_at_{timestamp}",




            )
            return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
        
    @property
    def _create_ckpt_callbacks(self):
            return tf.keras.callbacks.ModelCheckpoint(filepath=self.config.checkpoint_model_path,
                                                      save_best_only=True)
        

    def get_tb_ckpt_callbacks(self):
            return[
                self._create_tb_callbacks,
                self._create_ckpt_callbacks
            ]
             

In [59]:
try:
    config=ConfigurtaionManager()
    preparecallback_config=config.get_prepare_callbacks_config()
    preparecallbacks=PrepareCallback(config=preparecallback_config)
    callback_list=preparecallbacks.get_tb_ckpt_callbacks()


except Exception as e:
    raise e   

[2024-04-07 06:09:54,347: INFO: common: yaml file config\config.yaml load sucessfuly]
[2024-04-07 06:09:54,351: INFO: common: yaml file params.yaml load sucessfuly]
[2024-04-07 06:09:54,352: INFO: common: created dierctor ast artifacts]
[2024-04-07 06:09:54,354: INFO: common: created dierctor ast artifacts\prepare_callbacks\checkpoint_dir]
[2024-04-07 06:09:54,355: INFO: common: created dierctor ast artifacts\prepare_callbacks\tensorboard_log_dir]
